#  앙상블 Part 2 — Boosting & Gradient Boosting

---

### 목표
- Bagging(Random Forest)이 “**분산(Variance)**”을 줄이는 기법이었다면,
  이번엔 “**편향(Bias)**”을 줄이는 **Boosting**을 배운다.
- Boosting → AdaBoost → Gradient Boosting → XGBoost/LightGBM까지
  한 줄로 이해할 수 있는 **직관 + 코드 + 시각화**를 만든다.

In [1]:
# 팁
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

px.defaults.template = "plotly_dark"
px.defaults.width = 800
px.defaults.height = 450

RSEED = 42
np.random.seed(RSEED)


## Bagging vs Boosting 복습

---

### 개념 정리

| 구분 | **Bagging (배깅)** | **Boosting (부스팅)**            |
|------|--------------------|-------------------------------|
| 학습 방식 | **병렬(Parallel)** | **순차(Sequential)**            |
| 목표 | **Variance 감소 (안정성↑)** | **Bias 감소 (정확성↑)**            |
| 데이터 샘플링 | Bootstrap (복원추출) | Residual(잔차) 기반 반복 순차 학습      |
| 모델 간 관계 | 독립(Independent) | 종속(Dependent, 앞 모델 잔차(오차) 반영) |
| 대표 알고리즘 | Random Forest | AdaBoost, Gradient Boosting   |

### 정리
- **Bagging → 흔들림(Variance)을 줄이는 앙상블 (안정성 향상)**
- **Boosting → 오차(Bias)를 줄이는 앙상블 (정확성 향상)**


| **구분**       | **초점**            | **비유**       | **대표 알고리즘**   |
| ------------ | ----------------- | ------------ | ------------- |
| **Bagging**  | Variance ↓ (안정성↑) | 여러 명이 동시에 팀플 | Random Forest |
| **Boosting** | Bias ↓ (정확성↑)     | 오답노트로 단계적 보완 | AdaBoost, GBM |

## Boosting 아이디어 — 순차적 학습의 시작


### 핵심 아이디어
> 여러 개의 약한 모델이 순차적으로 학습**하면서,
> 앞 모델이 틀린 부분(오류, 잔차)**을 다음 모델이 보완**한다.

**핵심 로직**
1. 첫 번째 모델 → 데이터를 학습해 예측
2. 오차(Residual = 실제 - 예측) 계산
3. 다음 모델 → 이 오차(잔차)를 예측하도록 학습
4. 최종 예측 = 이전 예측 + 새로운 보정(오차 보정)


## AdaBoost — Adaptive Boosting

- 정의
    - AdaBoost는 틀린 샘플의 가중치를 올려, 다음 약한 모델(weak learner)이 어려운 샘플에 더 집중하게 만드는 순차형(Sequential) 앙상

- 학습 방법:
    1. 처음엔 모든 샘플 가중치가 동일 (w = 1/N)
	2. 틀린 샘플의 가중치 ↑ → 다음 모델이 그 샘플을 더 중요하게 학습
	3. 각 모델의 성능에 따라 모델 가중치(α_t) 부여
	4. 최종 예측 = 모델별 예측 × 가중치(α_t)의 합

##  참고
- 약한 학습기(weak learner): 보통 깊이 1 의사결정나무
- 핵심 하이퍼파라미터
    - n_estimators: 약한 모델의 개수 (너무 많으면 과적합 위험)
    - learning_rate: 한 단계 보정 강도(보폭). 작을수록 천천히, 안정적으로 → 더 많은 스텝 필요(시간이 더 걸림)

In [2]:
import numpy as np, pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

X, y = load_wine(return_X_y=True)
target_names = load_wine().target_names
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 약한 학습기: 깊이 1(Decision Stump)
stump = DecisionTreeClassifier(max_depth=1, random_state=42)

# 교차검증 설정(분류는 stratify가 기본)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### n_estimators 변화에 따른 성능 (learning_rate=0.5 고정)
- 왜? 스텝 수가 늘수록 점진적으로 좋아지다가 과적합될 수 있기 때문
- Test Accuracy + CV 평균을 함께 봄(운빨 대비)

In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import plotly.graph_objects as go
import pandas as pd

# 약한 학습기: 깊이 1 의사결정나무
stump = DecisionTreeClassifier(max_depth=1, random_state=42)

n_list = [10, 30, 50, 100, 200, 300]
rows = []

for n in n_list:
    ada = AdaBoostClassifier(
        estimator=stump,
        n_estimators=n,
        learning_rate=0.5,
        random_state=42,
    )
    ada.fit(X_train, y_train)
    test_acc = accuracy_score(y_test, ada.predict(X_test))
    cv_mean = cross_val_score(ada, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean()
    rows.append((n, test_acc, cv_mean))

df_curve = pd.DataFrame(rows, columns=["n_estimators", "Test Acc", "CV mean acc"])
display(df_curve)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_curve["n_estimators"], y=df_curve["Test Acc"],
    mode="lines+markers", name="Test Accuracy"
))
fig.add_trace(go.Scatter(
    x=df_curve["n_estimators"], y=df_curve["CV mean acc"],
    mode="lines+markers", name="CV mean Accuracy"
))
fig.update_layout(
    title="AdaBoost (learning_rate=0.5) — n_estimators vs Accuracy",
    xaxis_title="n_estimators",
    yaxis_title="Accuracy",
    template="plotly_dark",
    yaxis=dict(range=[0, 1])
)
fig.show()

# CV 평균과 Test값이 가깝게 **움직이면** 안정적임
# 멀어지면 오버핏 신호로 볼 수 있음
# 스탭수(n_es....) 너무 적으면 언더핏, 지나치게 많으면 오버핏으로 판단할 수 있음


,n_estimators,Test Acc,CV mean acc
0,10,0.916667,0.932381
1,30,0.972222,0.943651
2,50,0.972222,0.949206
3,100,0.972222,0.949206
4,200,0.972222,0.943651
5,300,0.972222,0.949365


### 해석
- 스텝 수(n_estimators)가 너무 적으면 언더핏(Underfitting), 지나치게 많으면 오버핏(Overfitting) 신호가 보일 수 있음
- CV 평균이 Test와 근접하면 안정적이고, 차이가 많이 벌어지면 과적합 의심

## (learning_rate × n_estimators) 그리드 히트맵 (CV 평균)
- 왜? 학습률–스텝 수 조합이 핵심 / 좋은 영역을 히트맵으로 한눈에

In [4]:
import plotly.figure_factory as ff

lr_list = [0.3, 0.2, 0.1, 0.05, 0.01]
n_list  = [30, 50, 100, 200, 300]

grid = []
for lr in lr_list:
    row = []
    for n in n_list:
        ada = AdaBoostClassifier(
            estimator=stump,
            n_estimators=n,
            learning_rate=lr,
            random_state=42,
        )
        cv_mean = cross_val_score(ada, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean()
        row.append(cv_mean)
    grid.append(row)

grid = np.array(grid)

fig = ff.create_annotated_heatmap(
    z=np.round(grid, 3),
    x=[str(n) for n in n_list],
    y=[str(lr) for lr in lr_list],
    colorscale="Viridis",
    showscale=True
)
fig.update_layout(
    title="AdaBoost — CV Accuracy Heatmap (learning_rate × n_estimators)",
    xaxis_title="n_estimators",
    yaxis_title="learning_rate",
    template="plotly_dark"
)
fig.show()

best_idx = np.unravel_index(np.argmax(grid), grid.shape)
best_lr = lr_list[best_idx[0]]
best_n  = n_list[best_idx[1]]
print(f"Best by CV: learning_rate={best_lr}, n_estimators={best_n}, CV acc={grid[best_idx]:.3f}")

Best by CV: learning_rate=0.3, n_estimators=100, CV acc=0.944


**해석**
- 일반적으로 **learning_rate가 작을수록** 더 천천히 안정적으로 좋아짐 → 그만큼 **더 많은 n_estimators** 필요
- 반대로 **learning_rate가 크면** 빠르게 좋아지지만 **과적합** 위험도 함께 증가
- 히트맵에서 **고원(plateau)**처럼 안정적이고 높은 구간을 선택하는게 합리적

### 최적 조합으로 최종 모델 학습 & 평가
- 정규화 혼동행렬(클래스별 비율) + 분류 리포트로 착시 제거
- 교차검증과 비교해 일관성 확인

In [5]:
ada_best = AdaBoostClassifier(
    estimator=stump,
    n_estimators=best_n,
    learning_rate=best_lr,
    random_state=42
)
ada_best.fit(X_train, y_train)

y_pred = ada_best.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"[AdaBoost Best] Test Accuracy: {test_acc:.3f}")
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))

cm_norm = confusion_matrix(y_test, y_pred, normalize="true")
fig = ff.create_annotated_heatmap(
    z=np.round(cm_norm, 3),
    x=list(target_names),
    y=list(target_names),
    colorscale="Blues",
    showscale=True
)
fig.update_layout(
    title="AdaBoost (Best) — Confusion Matrix (Normalized)",
    xaxis_title="Predicted",
    yaxis_title="Actual",
    template="plotly_dark"
)
fig.show()

[AdaBoost Best] Test Accuracy: 0.972
              precision    recall  f1-score   support

     class_0      0.923     1.000     0.960        12
     class_1      1.000     0.929     0.963        14
     class_2      1.000     1.000     1.000        10

    accuracy                          0.972        36
   macro avg      0.974     0.976     0.974        36
weighted avg      0.974     0.972     0.972        36



요약 & 실무 팁
- 핵심: AdaBoost는 틀린 샘플에 더 집중하는 순차 앙상블 → Bias(편향) 감소에 강함
- 약한 학습기는 보통 깊이 1 의사결정나무(Decision Stump)
- 학습률–스텝 조합이 성능/안정성 결정 → 히트맵/곡선으로 최적 범위를 찾기
- 과적합 신호: Test와 CV 간 간극↑, 혼동행렬에서 특정 클래스 오분류 급증
- (불균형 문제) 정확도만 보지 말고 F1/ROC-AUC/PR-AP도 함께 확인

## Gradient Boosting — Bias를 줄이는 두 번째 방법


### 목표
- AdaBoost가 **틀린 샘플(분류 오류)*에 집중했다면,
- Gradient Boosting은 오차의 방향(기울기)을 따라가며 Loss를 직접 줄이는 일반화된 알고리즘

### 핵심

1. 첫 번째 모델 → 데이터 전체를 대략 예측
2. 잔차(residual) = 실제값 − 예측값
3. 두 번째 모델 → 이 **잔차(residual)**를 예측
4. 전체 모델 = 이전 예측 + (학습률 × 잔차 보정)
5. 이 과정을 여러 번 반복하면 Loss가 점점 줄어듭니다.

즉, 각 스텝은 “이전 모델이 틀린 부분”을 따라가며 오차를 줄이는 경사하강법(Gradient Descent) 과정

## 차이

|**구분**|**AdaBoost**|**Gradient Boosting**|
|---|---|---|
|보정 기준|틀린 샘플의 가중치(weight) ↑|오차(loss function)의 기울기(gradient) ↓|
|업데이트 방식|분류 결과 기반|Loss function의 미분값 기반|
|주요 목표|어려운 샘플에 집중|Loss 자체를 최소화|
|대표 사용 분야|분류(Classification)|회귀·분류 모두 지원|
|대표 라이브러리|AdaBoostClassifier|GradientBoostingClassifier, XGBoost, LightGBM 등|


## 왜 Gradient(기울기)로 가야 할까?

- AdaBoost는 이진 분류 기반 구조라 Loss 형태가 제한적
- 하지만 Gradient Boosting은 **손실 함수(Loss Function)**를 설정할 수 있어서
- 회귀 문제, 다중 분류, 심지어 사용자 정의 Loss까지 모두 다룰 수 있음

## Gradient Descent 시각화

## 핵심 개념

기울기(Gradient)는 Loss가 얼마나 가파르게 변하는지를 알려주는 나침반

즉, 경사하강법은 기울기의 반대 방향으로 조금씩 이동하며 Loss를 최소화하는 지점을 찾는 방법

### **학습률(learning rate)의 의미**

|**구분**|**설명**|**결과**|
|---|---|---|
|**너무 작을 때**|천천히 움직임, 수렴은 하지만 시간이 오래 걸림|느림|
|**적절할 때**|안정적으로 최소점에 도달|이상적|
|**너무 클 때**|보폭이 커서 최소점을 지나치며 발산|불안정|


In [6]:
import numpy as np
import plotly.graph_objects as go

f = lambda x: x**2          # Loss 함수: f(x) = x²
df = lambda x: 2 * x        # 기울기: df/dx = 2x

x_vals = np.linspace(-6, 6, 400)
y_vals = f(x_vals)

def gradient_descent(start, lr, steps):
    x = start
    path = [(x, f(x))]
    for _ in range(steps):
        x -= lr * df(x)
        path.append((x, f(x)))
    return np.array(path)

paths = {
    "lr=0.01 (느림)": gradient_descent(5, 0.01, 80),
    "lr=0.1 (적절)": gradient_descent(5, 0.1, 20),
    "lr=0.3 (너무 큼)": gradient_descent(5, 0.3, 10)
}

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_vals, y=y_vals,
    mode="lines",
    name="Loss f(x)=x²",
    line=dict(color="gray", width=2)
))

for name, path in paths.items():
    fig.add_trace(go.Scatter(
        x=path[:, 0], y=path[:, 1],
        mode="markers+lines",
        name=name,
        marker=dict(size=6),
        line=dict(width=2)
    ))

fig.update_layout(
    template="plotly_dark",
    title="Gradient Descent — Learning Rate Comparison",
    xaxis_title="x (parameter)",
    yaxis_title="Loss f(x)",
    yaxis=dict(range=[0, 40]),
    legend=dict(title="Learning Rate", bgcolor="rgba(0,0,0,0)")
)

fig.show()

## Gradient Boosting 실습

In [7]:
from sklearn.datasets import load_wine
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingClassifier

wine = load_wine()
X, y = wine.data, wine.target
target_names = wine.target_names

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    "learning_rate": [0.3, 0.1, 0.05, 0.01],
    "n_estimators": [50, 100, 200, 300],
    # 필요시 함께 탐색 가능
    "subsample": [1.0, 0.8],
    "max_depth":[3, 2]
}

gb_base = GradientBoostingClassifier(max_depth=3, random_state=42)

gs = GridSearchCV(
    estimator=gb_base,
    param_grid=param_grid,
    scoring="accuracy",  # 불균형이면 'f1_macro' 등으로 교체
    cv=cv,
    n_jobs=-1,
    refit=True,
    return_train_score=True
)

gs.fit(X, y)

print("best_params_ :", gs.best_params_)
print("cv best_score:", gs.best_score_)

best_params_ : {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.8}
cv best_score: 0.9773015873015872


In [8]:
res = pd.DataFrame(gs.cv_results_)
display(res[[
    "param_learning_rate", "param_n_estimators",
    "mean_test_score", "std_test_score", "rank_test_score"
]].sort_values("rank_test_score").head(10))

heat = res.pivot_table(
    index="param_learning_rate",
    columns="param_n_estimators",
    values="mean_test_score"
).sort_index(ascending=False)

fig = ff.create_annotated_heatmap(
    z=np.round(heat.values, 3),
    x=heat.columns.astype(str).tolist(),
    y=heat.index.astype(str).tolist(),
    colorscale="Viridis",
    showscale=True
)
fig.update_layout(
    title="Gradient Boosting — CV Accuracy Heatmap (learning_rate × n_estimators)",
    xaxis_title="n_estimators",
    yaxis_title="learning_rate",
    template="plotly_dark"
)
fig.show()

,param_learning_rate,param_n_estimators,mean_test_score,std_test_score,rank_test_score
29,0.10,200,0.977302,0.033294,1
1,0.30,50,0.971746,0.031301,2
25,0.10,50,0.971746,0.031301,2
47,0.05,300,0.971746,0.031301,2
11,0.30,100,0.971746,0.031301,2
31,0.10,300,0.966190,0.028094,6
23,0.10,300,0.966190,0.033135,6
21,0.10,200,0.966190,0.033135,6
19,0.10,100,0.966190,0.033135,6
37,0.05,200,0.966190,0.033135,6


In [9]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import plotly.figure_factory as ff

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

gb_final = GradientBoostingClassifier(
    **gs.best_params_,
    random_state=42
).fit(X_train, y_train)

y_pred = gb_final.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"Final Test Accuracy: {test_acc:.3f}")
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))

cm_norm = confusion_matrix(y_test, y_pred, normalize="true")
fig = ff.create_annotated_heatmap(
    z=np.round(cm_norm, 3),
    x=list(target_names),
    y=list(target_names),
    colorscale="Blues",
    showscale=True
)
fig.update_layout(
    title="Gradient Boosting (Best) — Confusion Matrix (Normalized)",
    xaxis_title="Predicted", yaxis_title="Actual",
    template="plotly_dark"
)
fig.show()

Final Test Accuracy: 0.972
              precision    recall  f1-score   support

     class_0      0.923     1.000     0.960        12
     class_1      1.000     0.929     0.963        14
     class_2      1.000     1.000     1.000        10

    accuracy                          0.972        36
   macro avg      0.974     0.976     0.974        36
weighted avg      0.974     0.972     0.972        36



##  XGBoost / LightGBM 비교

| **알고리즘**                    | **핵심 포인트**                  | **한 줄 설명**                                  |
| --------------------------- | --------------------------- | ------------------------------------------- |
| **Gradient Boosting (GBM)** | 순차 트리 학습                    | 오차(잔차)를 줄이는 트리를 순차적으로 추가                    |
| **XGBoost**                 | 정규화(Regularization) + 병렬화   | GBM에 L1/L2 규제를 추가해 과적합을 억제하고, 병렬 학습으로 속도 향상 |
| **LightGBM**                | Leaf-wise 성장 + Histogram 기반 | 트리의 잎(leaf) 단위로 성장시켜 효율성 극대화 + 대용량 데이터에 최적화 |

| **구분**                 | **Gradient Boosting**                       | **XGBoost**           | **LightGBM**             |
| ---------------------- | ------------------------------------------- | --------------------- | ------------------------ |
| **학습 핵심**              | 순차 트리 학습                                    | GBM + L1/L2 정규화       | Leaf-wise 성장 + Histogram |
| **규제(Regularization)** | X                                           |  L1/L2 규제            | + Early Stopping       |
| **Tree 성장 방식**         | Depth-wise (균형 분할)                          | Depth-wise            | Leaf-wise (불균형, 성능↑)     |
| **병렬 처리**              | 제한적                                         | 가능 (Column 병렬)      | 가능 (Histogram 기반 더 빠름) |
| **메모리 효율**             | 낮음                                          | 중간                    | 높음 (Histogram 압축)        |
| **속도**                 | 중간                                          | 빠름                    | 매우 빠름                    |
| **대용량 데이터 처리**         | 어려움                                         | 보통                    | 매우 효율적                   |
| **대표 라이브러리**           | sklearn.ensemble.GradientBoostingClassifier | xgboost.XGBClassifier | lightgbm.LGBMClassifier  |

| **항목**       | **GBM** | **XGBoost** | **LightGBM**          |
| ------------ | ------- | ----------- | --------------------- |
| 속도        | 느림   | 빠름       | 매우 빠름              |
| 메모리 효율    | 낮음   | 중간       | 높음                 |
| 모델 복잡도 제어 | 없음    | 규제 추가     | 규제 + Early Stopping |
| 대용량 데이터   | 제한적     | 가능          | 매우 효율적                |

## 사용 예시(예시임)

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

gb = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
gb.fit(X_train, y_train)

xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    reg_lambda=1.0,
    n_jobs=-1,
    random_state=42,
    eval_metric="mlogloss"
)
xgb.fit(X_train, y_train)

lgb = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=-1,
    num_leaves=31,
    n_jobs=-1,
    random_state=42
)
lgb.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000169 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 498
[LightGBM] [Info] Number of data points in the train set: 142, number of used features: 13
[LightGBM] [Info] Start training from score -1.105679
[LightGBM] [Info] Start training from score -0.912776
[LightGBM] [Info] Start training from score -1.318241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,200
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


| **항목**                       | **보완 이유 / 팁**                                                                   |
| ---------------------------- | ------------------------------------------------------------------------------- |
| use_label_encoder=False      | 최신 XGBoost 버전에서 경고를 피하기 위해 설정                                                   |
| eval_metric="mlogloss"       | 다중 클래스 문제에서 손실 지표 명시 (XGBoost 기본 설정이 바뀔 수 있음)                                   |
| max_depth=-1 & num_leaves=31 | LightGBM은 “잎 기반(leaf-wise)” 성장 방식을 사용하므로, 깊이를 제한하는 대신 잎 개수를 제어                  |
| random_state=42 / n_jobs=-1  | 재현성 확보 + 병렬 처리 활용                                                               |
| **후속 평가**                    | 이후에 accuracy_score, classification_report, confusion_matrix 등을 비교해줘야 의미 전달이 완성됨 |

In [11]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[1, 2], y=[2, 1],
    mode="markers+text",
    text=["Bagging (Variance↓)", "Boosting (Bias↓)"],
    textposition="bottom center",
    marker=dict(size=40, color=["#00BFFF", "#FFB347"])
))

fig.add_annotation(x=1.5, y=1.5,
                   text="Bias–Variance 균형",
                   showarrow=False,
                   font=dict(size=14, color="white"))

fig.update_layout(
    title="앙상블의 두 축 — Bagging vs Boosting",
    xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
    yaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
    template="plotly_dark",
    width=600, height=400
)
fig.show()

| **구분**      | **Bagging**                  | **Boosting**                                   |
| ----------- | ---------------------------- | ---------------------------------------------- |
| **학습 방식**   | 병렬(Parallel) — 여러 모델이 동시에 학습 | 순차(Sequential) — 이전 모델 오차를 보완                  |
| **목표**      | **Variance 감소** (흔들림↓, 안정성↑) | **Bias 감소** (오차↓, 정밀도↑)                        |
| **데이터 샘플링** | Bootstrap (복원추출)             | Residual 기반 반복 학습                              |
| **모델 간 관계** | 독립 (각 모델이 별도 학습)             | 종속 (이전 모델 결과 반영)                               |
| **대표 알고리즘** | Random Forest                | AdaBoost, Gradient Boosting, XGBoost, LightGBM |
| **학습 비유**   | 여러 명이 동시에 팀플 수행              | 한 사람이 오답노트로 점진적 성장                             |